# Projeto 2 de Inteligência Artificial
### Disciplina: Introdução à Inteligência Artificial - UnB
### Turma: 01, 2025/2
### Professor: Dibio

### Integrantes:
#>- Adrielly Vitória Costa de Lima - 231018973

#>- Carlos Cauã Rocha da Silva - 231034304

#>- Leticia Gonçalves Bomfim - 241002411

O link da página do repositório é: https://github.com/Schatten900/proj2-iaa

## Aqui descreveremos as instruções de quais extensões devem ser baixadas e como rodar o projeto para Linux

1- Roda o de treinamento.py

2- Roda o ajustar_limiar.py

3- Se der um valor diferente do que consta no arquivo avaliar.py é só trocar o valor

4- Roda o avaliar.py para ele vai classificar as imagens

5- Roda o interpretar.py para ver os resultados e as fotos mostrando as diferenças

6- Roda a main.py

## Dataset, Interpretabilidade e Resultados_Inferência
   A estrutura apresentada no diretório do projeto foi organizada para seguir exatamente as etapas descritas no método de Katafuchi & Tokunaga (2020) proposto pelo trabalho, que propõe um diagnóstico de doenças em plantas utilizando detecção de anomalias não supervisionada baseada na reconstrução de cores por meio de um modelo Generative Adversarial Network (pix2pix).

   A pasta dataset contém os dados brutos fornecidos pelo professor e está dividida conforme solicitado no enunciado:

    Healthy_Train50 – 50 imagens de folhas saudáveis usadas para treinar o modelo generativo.

    Healthy_Test50 – 50 imagens saudáveis usadas para testar o sistema.

    Disease_Test100 – 100 imagens de folhas doentes empregadas na etapa final de inferência.

   Estas subdivisões permitem reproduzir o setup experimental do artigo original, que exige um modelo treinado com folhas saudáveis, de forma que qualquer falha na reconstrução indique a presença de anomalias.

   A pasta interpretabilidade reúne as figuras geradas durante as análises com Grad-CAM, conforme solicitado no projeto.
Arquivos como figura_0.png, figura_1.png, etc., mostram os mapas de ativação que destacam as regiões da folha que mais influenciam o modelo no processo de detecção de anomalia. Essa etapa permite verificar visualmente se a rede está focando nas áreas lesionadas.

  Por fim, a pasta resultados_inferencia contém as imagens geradas pelo modelo pix2pix e os resultados finais do método de reconstrução. Nela, cada folha original passa por três representações principais:

    real_leaf – é a imagem real da folha (saudável ou doente) usada como entrada.

    gerada_leaf – é a imagem reconstruída pelo modelo pix2pix, representando como a folha deveria ser se estivesse saudável, segundo o modelo treinado.

    delta_leaf – mapa de diferenças entre a imagem real e a reconstruída, revelando visualmente o índice de cor usado como métrica de anomalia no artigo. Quanto maior e mais concentrado o delta, maior a presença de doença.

Essa organização faz o treinamento apenas com folhas saudáveis, gera imagens de reconstruções com pix2pix, calcula do índice de cor para anomalia e gera a visualização interpretável da decisão do modelo (Grad-CAM). Assim, essa parte do código documenta claramente cada etapa executada, facilita testes adicionais e fornece o necessário para compor a análise de resultados exigida no projeto.

## Pasta Service
### Dataset.py

In [ ]:
import os
import tensorflow as tf
import cv2
import numpy as np

IMG_SIZE = 128


def carregar_imagem(caminho):
    """
    Carrega uma única imagem para inferência.
    Sem uso de tf.data, apenas cv2.
    """
    img = cv2.imread(caminho)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 127.5 - 1.0
    return img.astype("float32")

def carregar_imagem_cv2(caminho):
    caminho = caminho.decode("utf-8")
    img = cv2.imread(caminho)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 127.5 - 1.0
    return img.astype("float32")


def wrapper_processamento(caminho):
    img = tf.numpy_function(carregar_imagem_cv2, [caminho], tf.float32)
    img.set_shape([IMG_SIZE, IMG_SIZE, 3])
    return img, img


def criar_dataset_treino(pasta="dataset/Healthy_Train50", batch_size=2):

    caminhos = tf.data.Dataset.list_files(pasta + "/*", shuffle=True)

    dataset = caminhos.map(
        wrapper_processamento,
        num_parallel_calls=tf.data.AUTOTUNE
    )

    dataset = dataset.cache()
    dataset = dataset.shuffle(100)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset



   As funções carregar_imagem e carregar_imagem_cv2 executam, em sequência: Leitura e Conversão (carregam a imagem usando OpenCV (cv2) e convertem de BGR para RGB), Redimensionamento (ajustam todas as imagens para o tamanho fixo de 128 pixels (IMG_SIZE)) e Normalização (mapeiam os valores dos pixels para o intervalo [-1.0, 1.0] (fórmula: img / 127.5 - 1.0). Isso é crucial para o treinamento estável de Redes Neurais Generativas (como o Pix2Pix))
 
   Enquanto isso, a função criar_dataset_treino utiliza o TensorFlow dataset API para criar um pipeline de dados de alto desempenho:
•	wrapper_processamento: É o "pacote" que permite ao tf.data chamar a função OpenCV/NumPy para processamento, garantindo que o shape da imagem seja definido para (128, 128, 3).
•	return img, img: Prepara o dataset para o treinamento de um Autoencoder/Pix2Pix, onde a entrada (X) é a imagem saudável e a saída (Y) esperada é a própria imagem saudável (X=Y).
•	Otimização: A função implementa otimizações como processamento paralelo, caching, shuffling e prefetching para evitar gargalos de I/O durante o treinamento.

   Assim, o código prepara apenas o conjunto de treinamento, composto pelas 50 imagens de folhas saudáveis. Durante o teste, a folha doente apresentará um alto erro de reconstrução no pix2pix, que será o indicador da anomalia, conforme o método do artigo de Katafuchi & Tokunaga.

### Discriminador.py

   A arquitetura do Discriminador é baseada no modelo PatchGAN, usado em Pix2Pix, onde ele tenta classificar se uma região da imagem é real ou gerado, em vez de classificar a imagem inteira. O Discriminador de um Pix2Pix recebe duas imagens: a imagem de entrada (inp - a folha original) e a imagem alvo (tar - a imagem que o Gerador produziu, que neste contexto seria a folha reconstruída).

Geração do Grad-CAM no Discriminador:
   A função gerar_gradcam_discriminador implementa o algoritmo Grad-CAM,usado para visualização e coloca em prática que:
    1.	O modelo é treinado em folhas saudáveis.
    2.	Quando uma folha doente é testada, o Discriminador tende a dar uma pontuação baixa para a combinação (Folha Original, Folha Reconstruída).
    3.	O Grad-CAM identifica quais pixels ou regiões da imagem mais influenciaram a decisão do Discriminador.
    
   Como entradas recebe o Discriminador já treinado, a imagem de entrada (img_in) e a imagem reconstruída (img_out), faz a identificação da camada, onde o código encontra a última camada convolucional do Discriminador (ultima_conv), cria um modelo temporário que tem a saída da última camada convolucional E a saída final do Discriminador e calcula o gradiente da saída  em relação aos mapas de características (conv_out) da última camada convolucional. O gradiente indica a importância de cada feature map para a decisão final.
   
   Já o cálculo dos pesos (grads) é feito com o gradiente, que é globalmente agrupado (tf.reduce_mean) para obter os pesos médios de importância de cada canal de feature map. A geração do mapa de ativação multiplica os pesos médios (grads) pelos mapas de características (conv_out) e soma-os (tf.reduce_sum).
   
   Por fim, aplica uma função de ativação ReLU (np.maximum(cam, 0)) para focar apenas nas contribuições positivas e, em seguida, normaliza o resultado (entre 0 e 1). Como resultado temos que o cam é um mapa de calor que mostra as regiões da folha que o Discriminador considerou mais importantes.
   
### Gerador.py

   Este código define a arquitetura do Gerador da Rede Adversária Generativa (GAN) do projeto, utilizando a estrutura U-Net. O Gerador é responsável por receber uma folha original e produzir a folha reconstruída, ou seja, saudável.
   
   O Gerador foi pensado como uma arquitetura de Autoencoder com Conexões de Salto, o que define a U-Net:
•	Encoder (d1 a d6): Reduz progressivamente a imagem de entrada até um gargalo (bottleneck) de 4 por 4 pixels.
•	Decoder (u1 a u5): Aumenta o tamanho espacial do gargalo de volta para 128 por 128.
•	layers.Concatenate(): Esta é a característica central da U-Net. Em cada etapa do Decoder (u1 a u5), a saída é concatenada com o mapa de características correspondente do Encoder (d5 a d1). Isso permite que informações de alta resolução e detalhes finos (perdidos no gargalo) sejam passadas diretamente para a fase de reconstrução.
•	Saída: A camada final Conv2DTranspose com ativação tanh produz a imagem reconstruída. A função mapeia a saída para o intervalo [-1.0, 1.0], que corresponde à normalização aplicada na entrada (carregar_imagem).

   O Gerador é treinado para atuar como um Autoencoder Identidade para folhas saudáveis. Sua função é realizar o treinamento, recebendo uma folha saudável e aprendendo a reconstruí-la perfeitamente. E testar, recebendo uma folha doente e falhando na reconstrução das áreas afetadas, produzindo uma imagem que se parece com uma folha saudável. A diferença entre a entrada doente e a saída reconstruída é o mapa de anomalias que será usado para o diagnóstico.

### Gradcam.py

In [ ]:
import tensorflow as tf
import numpy as np


def encontrar_ultima_conv(modelo):
    """
    Percorre o modelo inteiro, inclusive blocos Sequential,
    para encontrar a última camada Conv2D.
    """
    ultima_conv = None

    def explorar(layers):
        nonlocal ultima_conv
        for layer in layers:
            # se for Conv2D
            if isinstance(layer, tf.keras.layers.Conv2D):
                ultima_conv = layer

            # se for um bloco Sequential, explorar dentro
            if isinstance(layer, tf.keras.Model) or isinstance(layer, tf.keras.Sequential):
                explorar(layer.layers)

    explorar(modelo.layers)
    return ultima_conv


def gerar_gradcam(modelo, img_normalizada):

    entrada = tf.expand_dims(img_normalizada, axis=0)

    # achar a última camada conv2D
    ultima_conv = encontrar_ultima_conv(modelo)
    if ultima_conv is None:
        raise ValueError("Nenhuma camada Conv2D encontrada no modelo (nem dentro de Sequential).")

    # criar modelo que mapeia entrada -> (featuremap, saída final)
    grad_model = tf.keras.models.Model(
        inputs=modelo.inputs,
        outputs=[ultima_conv.output, modelo.output]
    )

    with tf.GradientTape() as tape:
        conv_out, pred = grad_model(entrada)
        loss = tf.reduce_mean(pred)

    grads = tape.gradient(loss, conv_out)
    weights = tf.reduce_mean(grads, axis=(0, 1, 2))
    cam = tf.reduce_sum(weights * conv_out[0], axis=-1)

    cam = np.maximum(cam, 0)
    cam = cam / (np.max(cam) + 1e-8)

    return cam

   Essa parte do código diz respeito à implementação do algoritmo Grad-CAM, projetado para funcionar em qualquer modelo sequencial ou com blocos internos, como o Discriminador ou Gerador definidos anteriormente.
Encontra a última camada convolucional, usando uma abordagem de recursão (explorar(layers)) para atravessar a estrutura de camadas do modelo, inclusive explorando dentro de blocos definidos como Sequential ou Model. Assim, garante que a última Conv2D de todo o grafo seja identificada.

   A função gerar_gradcam aplica o algoritmo do Grad-CAM. Ou seja, o modelo de gradiente (grad_model, cria um modelo auxiliar que tem duas saídas, sendo eles os mapas de características da ultima_conv e a saída final de predição do modelo (modelo.output). Além disso, utiliza tf.GradientTape() para calcular o gradiente da saída final (loss = tf.reduce_mean(pred)) em relação aos mapas de características da ultima_conv (conv_out). Este gradiente indica a importância relativa de cada mapa de característica para a predição.
   
   Já o cálculo dos pesos usa o gradiente globalmente agrupado (tf.reduce_mean) para obter o peso de importância de cada canal de feature map. Enquanto o mapa de calor (cam) é obtido pela multiplicação pontual dos pesos pelos mapas de características (conv_out) e somando os resultados ao longo do eixo dos canais.
   
   Para a normalização, aplicamos a função de ativação ReLU (np.maximum(cam, 0)) para remover contribuições negativas e normaliza o mapa final para o intervalo [0, 1].
   
   Dessa forma, a visualização pode ocorrer tanto no discriminador quanto no gerador. No Discriminador identifica as regiões que mais convenceram o discriminador de que a reconstrução é saudável e no Gerador pode ser usado para ver quais regiões da imagem de entrada o Gerador está usando para produzir a saída reconstruída.
   
### Metrics.py

![Exemplo](interpretabilidade/figura_0.png)


![Exemplo](interpretabilidade/figura_1.png)


![Exemplo](interpretabilidade/figura_2.png)


![Exemplo](interpretabilidade/figura_3.png)

In [ ]:
import numpy as np
from skimage import color

def mapa_deltaE_ciede2000(real, gerada):
    #normaliza para [0,1
    if real.dtype == np.uint8:
        real = real.astype("float32") / 255.0
    if gerada.dtype == np.uint8:
        gerada = gerada.astype("float32") / 255.0
    
    lab_real = color.rgb2lab(real)
    lab_gerada = color.rgb2lab(gerada)

    deltaE = color.deltaE_ciede2000(lab_real, lab_gerada)
    
    return deltaE

   O sistema começa com o pré-processamento e carregamento de dados eficiente, onde as imagens de folhas saudáveis são lidas, redimensionadas para 128 por 128 pixels e normalizadas para o intervalo [-1.0, 1.0]. O dataset é preparado para alimentar o modelo com a imagem de entrada e a imagem alvo idênticas (X=Y).
   
   A arquitetura do sistema é composta por duas redes principais: o Gerador e o Discriminador. O Gerador que serve como um autoencoder que comprime e depois descompacta a imagem, mantendo a fidelidade estrutural através das conexões diretas entre o encoder e o decoder. Enquanto o Discriminador que força o Gerador a produzir resultados realistas.
   
   Finalmente, o sistema inclui ferramentas essenciais de avaliação e visualização. A função mapa_deltaE_ciede2000 implementa a métrica, uma medida de diferença de cor que será usada para gerar o mapa de anomalias, onde altos valores de entre a folha doente original e a reconstrução saudável indicam a área da doença. Para a visualização, o código fornece uma implementação do Grad-CAM, capaz de identificar as regiões exatas da imagem que mais influenciaram a decisão do Discriminador, fornecendo uma explicação visual para o diagnóstico. Em conjunto, o código estabelece um pipeline desde o carregamento otimizado dos dados até a construção do modelo e as ferramentas de avaliação exigidas pelo projeto.

### Ajustar_limiar.py

In [ ]:
import glob
import numpy as np
from inferencia import carregar_modelo, desnormalizar
from services.dataset import carregar_imagem
from services.metrics import mapa_deltaE_ciede2000

def ajustar():

    saudaveis = glob.glob("dataset/Healthy_Test50/*")
    doentes = glob.glob("dataset/Disease_Test100/*")

    gerador = carregar_modelo()

    erros_saudaveis = []
    erros_doentes = []

    for img_path in saudaveis:
        real_norm = carregar_imagem(img_path)
        real = desnormalizar(real_norm)
        fake = gerador(np.expand_dims(real_norm,0))[0].numpy()
        fake = desnormalizar(fake)
        deltaE = mapa_deltaE_ciede2000(real, fake)
        erros_saudaveis.append(np.mean(deltaE))

    for img_path in doentes:
        real_norm = carregar_imagem(img_path)
        real = desnormalizar(real_norm)
        fake = gerador(np.expand_dims(real_norm,0))[0].numpy()
        fake = desnormalizar(fake)
        deltaE = mapa_deltaE_ciede2000(real, fake)
        erros_doentes.append(np.mean(deltaE))

    print("\nMédias:")
    print("Saudáveis:", np.mean(erros_saudaveis))
    print("Doentes  :", np.mean(erros_doentes))

    print("\nSugestão de limiar:")
    print((np.mean(erros_saudaveis)+np.mean(erros_doentes))/2)

if __name__ == "__main__":
    ajustar()


![Saída](imagens/limiar.png)

   Este código finaliza o pipeline do projeto, implementando a etapa de avaliação do modelo treinado e a definição do limiar de detecção de anomalias. Ele testa o Gerador nos conjuntos de teste de folhas saudáveis e doentes.
   
   O código itera sobre os dois conjuntos de teste, sendo eles saudáveis e doentes, para calcular o Erro de Reconstrução médio de cada imagem. Para cada imagem segue o seguinte fluxo: Carregamento e Normalização, onde o real_norm = carregar_imagem(img_path) carrega e normaliza a folha original para o intervalo [-1.0, 1.0]. A seguir, temos a geração por fake = gerador(np.expand_dims(real_norm,0))[0].numpy() que passa a folha original pelo gerador para obter a folha reconstruída. E, por fim, a imagem real (real) e a imagem reconstruída (fake) são desnormalizadas para o intervalo [0, 1] ou [0, 255] para serem compatíveis com o cálculo da métrica de cor. Já o cálculo de métrica se dá por deltaE = mapa_deltaE_ciede2000(real, fake), que calcula o mapa de diferença de cor perceptível  entre a folha original e a reconstrução e o erro médio por np.mean(deltaE), que calcula o erro de reconstrução médio para a imagem inteira.
   
### Avaliar.py

In [ ]:
import glob
import numpy as np

from inferencia import carregar_modelo, desnormalizar
from services.dataset import carregar_imagem
from services.metrics import mapa_deltaE_ciede2000

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

LIMIAR = 3.325781

def avaliar():

    saudaveis = glob.glob("dataset/Healthy_Test50/*")
    doentes   = glob.glob("dataset/Disease_Test100/*")

    print(f"Saudáveis: {len(saudaveis)}")
    print(f"Doentes: {len(doentes)}")

    if len(saudaveis) == 0 and len(doentes) == 0:
        print("Nenhuma imagem encontrada.")
        return

    gerador = carregar_modelo()

    y_true = []
    y_pred = []

    todos = saudaveis + doentes
    labels = [0]*len(saudaveis) + [1]*len(doentes)

    for img_path, true_label in zip(todos, labels):

        real_norm = carregar_imagem(img_path)
        real = desnormalizar(real_norm)

        fake_norm = gerador(np.expand_dims(real_norm, axis=0))[0].numpy()
        fake = desnormalizar(fake_norm)

        deltaE = mapa_deltaE_ciede2000(real, fake)
        erro = np.mean(deltaE)

        pred = 1 if erro > LIMIAR else 0

        y_true.append(true_label)
        y_pred.append(pred)

    if len(y_true) == 0:
        print("Nenhuma imagem válida processada.")
        return

    print("\n MÉTRICAS:\n")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, zero_division=0))
    print("Recall   :", recall_score(y_true, y_pred, zero_division=0))
    print("F1-score :", f1_score(y_true, y_pred, zero_division=0))

if __name__ == "__main__":
    avaliar()

![Saída](imagens/avaliar.png)

   Este parte do código implementa a etapa de avaliação final de desempenho do sistema de diagnóstico de anomalias, transformando o erro de reconstrução em uma decisão de classificação binária e calculando as métricas de desempenho mais relevantes.
   
   O código usa um valor de limiar fixo, presumivelmente determinado na etapa anterior, que separou o erro de reconstrução médio das imagens saudáveis e doentes. E, também carrega todos os caminhos das imagens de teste (50 saudáveis, 100 doentes) e cria a lista de rótulos verdadeiros (y_true), onde 0 significa Saudável e 1 significa Doente.
   
   Após isso, o código itera sobre todas as 150 imagens de teste. Cada imagem é processada pelo Gerador, gerando sua reconstrução saudável e também é calculado o erro médio de cor entre a imagem original e a reconstrução, esse erro é comparado com o limiar e se erro > 3.325781, a imagem é classificada como doente (pred = 1) ou se erro <= 3.325781, a imagem é classificada como Saudável (pred = 0). Por fim, o rótulo verdadeiro (true_label) e a predição (pred) são armazenados em y_true e y_pred.
   
### Carregar_Modelo.py

   Essa parte do código é responsável por reconstruir o modelo gerador e carregar nele os pesos previamente treinados. Ela começa verificando se o arquivo indicado, que armazena os pesos do modelo, realmente existe. Caso o arquivo não seja encontrado no diretório especificado, a função interrompe o processo e informa ao usuário que o arquivo de pesos não está disponível, evitando erros posteriores de execução.
   
   Se o arquivo estiver presente, a função prossegue chamando construir_gerador, que monta a arquitetura do modelo gerador utilizada no sistema. Em seguida, o modelo é explicitamente construído com uma forma de entrada padrão de 128×128 pixels e três canais de cor, garantindo que sua estrutura esteja completamente inicializada antes do carregamento dos pesos.
   
   Após essa preparação, os pesos salvos no arquivo são carregados no modelo, restaurando o estado aprendido durante o treinamento. Quando o processo é concluído com sucesso, uma mensagem é exibida ao usuário confirmando que o carregamento foi realizado corretamente. Por fim, a função retorna o modelo já configurado e pronto para ser usado nas etapas de inferência ou avaliação.
   
### Inferência.py

In [ ]:
# inferir.py
#
# Carrega o gerador salvo em "gerador_treinado.h5"
# e realiza inferência e classificação.

import os
import cv2
import numpy as np
import tensorflow as tf

from services.dataset import carregar_imagem
from services.metrics import mapa_deltaE_ciede2000
from services.gerador import construir_gerador



def carregar_modelo(path="gerador_treinado.h5"):
    return tf.keras.models.load_model(path, compile=False)

def desnormalizar(img):
    img = (img + 1) * 127.5
    return np.clip(img, 0, 255).astype(np.uint8)
    
def inferir(img_path, limiar=25):

    gerador = carregar_modelo()

    real_norm = carregar_imagem(img_path)
    real = desnormalizar(real_norm)

    fake_norm = gerador(np.expand_dims(real_norm, axis=0))[0].numpy()
    fake = desnormalizar(fake_norm)

    deltaE = mapa_deltaE_ciede2000(real, fake)
    erro = np.mean(deltaE)

    classe = "Folha DOENTE" if erro > limiar else "Folha SAUDÁVEL"

    os.makedirs("resultados_inferencia", exist_ok=True)
    nome = os.path.basename(img_path)

    cv2.imwrite(f"resultados_inferencia/real_{nome}", cv2.cvtColor(real, cv2.COLOR_RGB2BGR))
    cv2.imwrite(f"resultados_inferencia/gerada_{nome}", cv2.cvtColor(fake, cv2.COLOR_RGB2BGR))
    cv2.imwrite(f"resultados_inferencia/deltaE_{nome}", deltaE)

    print("Classificação:", classe)
    print("Erro médio:", erro)

    return classe

def carregar_modelo_pesos(caminho="gerador_treinado.weights.h5"):
    if not os.path.exists(caminho):
        print("Erro: pesos não encontrados!")
        return None
    
    modelo = construir_gerador()
    modelo.build((None, 128, 128, 3))
    modelo.load_weights(caminho)
    print("Pesos carregados com sucesso!")
    return modelo

   Essa parte é responsável pela etapa de inferência do sistema, isto é, o momento em que o modelo já treinado é utilizado para analisar novas imagens de folhas e decidir se elas estão saudáveis ou doentes, nele, a reconstrução de uma folha saudável serve como referência para identificar anomalias na imagem original.
   
   Há duas funções para carregar o modelo treinado, uma utiliza diretamente um arquivo .h5 contendo toda a estrutura e pesos do modelo, enquanto a outra reconstrói a arquitetura a partir do zero e então carrega apenas os pesos salvos. Ambas as abordagens permitem restaurar o estado aprendido durante o treinamento. A função desnormalizar (desnormalizar(img)), tem função de converter imagens que estão no intervalo [-1, 1] de volta para o formato tradicional de pixels entre 0 e 255.
   
   Já a função central (inferir(img_path, limiar=25)), tem função de carregar e normalizar a imagem de entrada como mesma feito no treinamento. Depois disso, o modelo gerador produz uma reconstrução da imagem, representando a versão ideal dessa folha. Tanto a imagem real quanto a reconstruída passam pela desnormalização para que possam ser visualizadas e comparadas adequadamente.
   
   A comparação entre as duas imagens é feita através da métrica DeltaE CIEDE2000, desse cálculo resulta um mapa de diferenças entre real e reconstruída, além de um valor médio que sintetiza o quão discrepante a imagem original é da versão saudável produzida pelo gerador. Quanto maior esse erro médio, maior a probabilidade de que a folha apresente algum tipo de doença ou anomalia. A classificação é feita comparando esse erro médio com um limiar previamente ajustado. Quando o erro ultrapassa o limiar, a folha é considerada doente; quando está abaixo, é classificada como saudável. Esse limiar é calculado em outra etapa do projeto, justamente para permitir uma separação mais precisa entre as duas classes.
   
   Ao fim, a pasta chamada resultados_inferencia salva três arquivos, sendo eles a imagem real, a reconstrução gerada pelo modelo e o mapa DeltaE que destaca as diferenças entre ambas. O script informa no terminal qual foi a classe atribuída à imagem e qual foi o valor do erro médio encontrado, além de retornar o resultado da classificação. Dessa forma, o arquivo inferir.py encapsula todo o processo de diagnóstico, desde o carregamento do modelo até a geração das imagens e dos indicadores.

### Interpretar.py

In [ ]:
import os
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

from carregar_modelo import carregar_modelo_pesos
from services.dataset import carregar_imagem
from services.metrics import mapa_deltaE_ciede2000
from services.discriminador import gerar_gradcam_discriminador
from services.discriminador import construir_discriminador
from inferencia import desnormalizar


os.makedirs("interpretabilidade", exist_ok=True)


def interpretar_imagem(caminho_img, gerador, idx):

    # imagem real normalizada [-1,1]
    real_norm = carregar_imagem(caminho_img)
    real = desnormalizar(real_norm)

    entrada = np.expand_dims(real_norm, axis=0)

    # reconstrução
    fake_norm = gerador(entrada, training=False)[0].numpy()
    fake = desnormalizar(fake_norm)

    # ΔE2000
    deltaE = mapa_deltaE_ciede2000(real, fake)

    # Grad-CAM do DISCRIMINATOR
    discriminador = construir_discriminador()
    cam = gerar_gradcam_discriminador(discriminador, real_norm, fake_norm)
    cam = cv2.resize(cam, (128, 128))

    # heatmap
    heatmap = cv2.applyColorMap((cam * 255).astype("uint8"), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(heatmap, 0.5, real, 0.5, 0)

    fig, ax = plt.subplots(1, 5, figsize=(20, 4))

    ax[0].imshow(real)
    ax[0].set_title("Original")
    ax[0].axis("off")

    ax[1].imshow(fake)
    ax[1].set_title("Reconstruída")
    ax[1].axis("off")

    ax[2].imshow(deltaE, cmap="inferno")
    ax[2].set_title("ΔE2000")
    ax[2].axis("off")

    ax[3].imshow(cam, cmap="jet")
    ax[3].set_title("Grad-CAM (Discriminador)")
    ax[3].axis("off")

    ax[4].imshow(overlay)
    ax[4].set_title("Overlay")
    ax[4].axis("off")

    nome = f"interpretabilidade/figura_{idx}.png"
    plt.savefig(nome, dpi=200, bbox_inches="tight")
    plt.close()

    print(f"✔ Figura salva: {nome}")


def executar_interpretabilidade():

    gerador = carregar_modelo_pesos()

    imagens = (
        glob.glob("dataset/Healthy_Test50/*")[:2] +
        glob.glob("dataset/Disease_Test100/*")[:2]
    )

    print(f"\nRodando interpretabilidade para {len(imagens)} imagens...\n")

    for i, img in enumerate(imagens):
        interpretar_imagem(img, gerador, i)


if __name__ == "__main__":
    executar_interpretabilidade()


![Saída](imagens/interpretar.png)

   Já nessa parte do código, passa a lidar com um segundo modo de carregamento do gerador, permitindo que o modelo seja reconstruído manualmente antes de receber apenas os pesos treinados. Para isso, o código verifica se o arquivo contendo os pesos realmente existe e, caso contrário, interrompe o processo para evitar falhas. 
   
   Quando o arquivo é encontrado, o gerador é montado a partir da função de construção já utilizada no treinamento e depois inicializado com as dimensões esperadas, garantindo compatibilidade na hora de aplicar os pesos. Após essa preparação, o carregamento é realizado e uma confirmação é exibida, indicando que o modelo está pronto para ser usado em inferências. Essa abordagem oferece flexibilidade adicional ao projeto, permitindo tanto o carregamento do modelo completo quanto apenas dos pesos, dependendo da necessidade.
   
### Main.py

In [ ]:
# main.py
#
# Menu principal para:
#   1. Treinar o modelo
#   2. Realizar inferência em imagem aleatória do dataset de teste
#

import os
import random

from treinamento import treinar
from inferencia import inferir

# Pastas do dataset de teste
PASTAS_TESTE = [
    "dataset/Disease_Test100",
    "dataset/Healthy_Test50"
]


def escolher_imagem_aleatoria():
    """
    Seleciona uma imagem aleatória do dataset de teste.
    """
    pasta = random.choice(PASTAS_TESTE)

    imagens = [
        arq for arq in os.listdir(pasta)
        if arq.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    if not imagens:
        raise RuntimeError(f"Nenhuma imagem encontrada na pasta {pasta}")

    nome = random.choice(imagens)
    caminho = os.path.join(pasta, nome)

    print(f"\n🌿 Imagem sorteada para inferência: {caminho}\n")
    return caminho


def executar_inferencia():
    imagem = escolher_imagem_aleatoria()
    resultado = inferir(imagem)   # <-- removido checkpoint_dir
    print(f"\nResultado Final da Inferência: {resultado}\n")



def executar_treinamento():
    """
    Executa o treinamento do modelo.
    """
    print("\nIniciando treinamento...\n")
    treinar()
    print("\nTreinamento finalizado!\n")


def main():
    while True:
        print("\n==============================")
        print("     SISTEMA PIX2PIX IA")
        print("==============================")
        print("1 - Treinar modelo")
        print("2 - Inferir imagem aleatória do teste")
        print("3 - Sair")
        print("==============================\n")

        opcao = input("Escolha uma opção: ")

        if opcao == "1":
            executar_treinamento()

        elif opcao == "2":
            executar_inferencia()

        elif opcao == "3":
            print("Encerrando...")
            break

        else:
            print("Opção inválida!")
    exit()

if __name__ == "__main__":
    main()

   O código main.py é o menu principal e o ponto de entrada do seu sistema de diagnóstico de folhas, contendo a interface de controle. Sua função principal é permitir ao usuário escolher entre treinar o modelo (treinar(), importado) ou realizar inferência (inferir(), importado).
   
   A função escolher_imagem_aleatoria é responsável por sortear, de forma equilibrada, uma imagem aleatória tanto do conjunto de folhas saudáveis quanto do conjunto de folhas doentes para ser usada no teste imediato de inferência.
   
   As funções executar_treinamento e executar_inferencia chamam os módulos de treinamento e inferência, respectivamente, garantindo que o fluxo do programa seja mantido. A função main orquestra este menu em um loop contínuo até que o usuário escolha sair.
   
### Treinamento.py

In [ ]:

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import losses

from services.dataset import criar_dataset_treino
from services.gerador import construir_gerador
from services.discriminador import construir_discriminador


LR = 2e-4
LAMBDA_L1 = 50     # menor = treinamento mais rápido
EPOCHS = 60
BATCH_SIZE = 2


def perda_discriminador(real_output, fake_output):
    BCE = losses.BinaryCrossentropy(from_logits=True)
    return 0.5 * (BCE(tf.ones_like(real_output), real_output) +
                  BCE(tf.zeros_like(fake_output), fake_output))


def perda_gerador_gan(fake_output):
    BCE = losses.BinaryCrossentropy(from_logits=True)
    return BCE(tf.ones_like(fake_output), fake_output)


def perda_l1(real, fake):
    return tf.reduce_mean(tf.abs(real - fake))


opt_g = Adam(LR, beta_1=0.5)
opt_d = Adam(LR, beta_1=0.5)


@tf.function
def treinar_batch(gerador, discriminador, real_in, real_tgt):

    with tf.GradientTape() as tape_g, tf.GradientTape() as tape_d:

        fake_img = gerador(real_in, training=True)

        disc_real = discriminador([real_in, real_tgt], training=True)
        disc_fake = discriminador([real_in, fake_img], training=True)

        g_gan = perda_gerador_gan(disc_fake)
        g_l1 = perda_l1(real_tgt, fake_img)

        g_total = g_gan + LAMBDA_L1 * g_l1

        d_total = perda_discriminador(disc_real, disc_fake)

    grad_g = tape_g.gradient(g_total, gerador.trainable_variables)
    grad_d = tape_d.gradient(d_total, discriminador.trainable_variables)

    opt_g.apply_gradients(zip(grad_g, gerador.trainable_variables))
    opt_d.apply_gradients(zip(grad_d, discriminador.trainable_variables))

    return g_total, d_total


def treinar():

    dataset = criar_dataset_treino(batch_size=BATCH_SIZE)

    gerador = construir_gerador()
    discriminador = construir_discriminador()

    print("\n=========== INICIANDO TREINAMENTO ===========")

    for ep in range(1, EPOCHS + 1):

        g_loss_ep = 0
        d_loss_ep = 0
        batches = 0

        for real_in, real_tgt in dataset:
            g_loss, d_loss = treinar_batch(gerador, discriminador, real_in, real_tgt)
            g_loss_ep += g_loss
            d_loss_ep += d_loss
            batches += 1

        print(f"Época {ep}/{EPOCHS} | "
              f"G_Loss = {g_loss_ep/batches:.4f} | "
              f"D_Loss = {d_loss_ep/batches:.4f}")

    print("\n=========== TREINAMENTO FINALIZADO ===========\n")

    gerador.save("gerador_treinado.h5")
    print("Modelo salvo como gerador_treinado.h5")

    return gerador


if __name__ == "__main__":
    treinar()

![Saída](imagens/treinamento.png)

   Essa parte do código é responsável pelo treinamento. O código apresentado é o módulo Treinamento (treinar.py), que define e executa o processo de aprendizado da GAN, responsável por gerar as reconstruções de folhas saudáveis. O treinamento é regido por parâmetros chave como a taxa de aprendizado e o fator de peso da perda de reconstrução, crucial para forçar a fidelidade da imagem gerada.
   
   A dinâmica do treinamento é centralizada na função treinar_batch, onde o gerador cria uma imagem falsa, em seguida, o discriminador avalia a imagem falsa e a imagem real. As perdas são então calculadas. A perda total do gerador é uma combinação da perda adversarial e da perda L1. Por sua vez, a perda do discriminador é a soma da sua capacidade de classificar corretamente os pares reais e os pares falsos, usando gradientes calculados via tf.GradientTape, os otimizadores Adam separados que atualizam os pesos de cada rede de forma competitiva. 
   
   O loop principal treinar esse processo, iterando sobre batches do dataset de folhas saudáveis. Ao final, o gerador treinado é salvo no arquivo gerador_treinado.h5, tornando-o pronto para ser usado na inferência e detecção de anomalias.